### Multi-Agent Systems

Multi-agent systems enable specialized agents to collaborate on complex tasks, improving modularity, scalability, and robustness. Instead of relying on a single agent, tasks are distributed among agents with distinct capabilities.

In smolagents, different agents can be combined to generate Python code, call external tools, perform web searches, and more. By orchestrating these agents, we can create powerful workflows.

A typical setup might include:

A Manager Agent for task delegation
A Code Interpreter Agent for code execution
A Web Search Agent for information retrieval
The diagram below illustrates a simple multi-agent architecture where a Manager Agent coordinates a Code Interpreter Tool and a Web Search Agent, which in turn utilizes tools like the DuckDuckGoSearchTool and VisitWebpageTool to gather relevant information.


Multi-Agent Systems in Action
A multi-agent system consists of multiple specialized agents working together under the coordination of an Orchestrator Agent. This approach enables complex workflows by distributing tasks among agents with distinct roles.

For example, a Multi-Agent RAG system can integrate:

A Web Agent for browsing the internet.
A Retriever Agent for fetching information from knowledge bases.
An Image Generation Agent for producing visuals.
All of these agents operate under an orchestrator that manages task delegation and interaction.

Solving a complex task with a multi-agent hierarchy
You can follow the code in this notebook that you can run using Google Colab.

The reception is approaching! With your help, Alfred is now nearly finished with the preparations.

But now there’s a problem: the Batmobile has disappeared. Alfred needs to find a replacement, and find it quickly.

Fortunately, a few biopics have been done on Bruce Wayne’s life, so maybe Alfred could get a car left behind on one of the movie sets, and re-engineer it up to modern standards, which certainly would include a full self-driving option.

But this could be anywhere in the filming locations around the world - which could be numerous.

So Alfred wants your help. Could you build an agent able to solve this task?

👉 Find all Batman filming locations in the world, calculate the time to transfer via boat to there, and represent them on a map, with a color varying by boat transfer time. Also represent some supercar factories with the same boat transfer time.

Let’s build this!

This example needs some additional packages, so let’s install them first:

In [3]:
! pip install smolagents[litellm] plotly geopandas shapely kaleido -q

<B> We first make a tool to get the cargo plane transfer time. </B>

In [1]:
import math
from typing import Optional, Tuple

from smolagents import tool

@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0, # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)
    
    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )

    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1


    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


### Setting up the agent
For the model provider, we use Together AI, one of the new inference providers on the Hub!

The GoogleSearchTool uses the Serper API to search the web, so this requires either having setup env variable SERPAPI_API_KEY and passing provider="serpapi" or having SERPER_API_KEY and passing provider=serper.

If you don’t have any Serp API provider setup, you can use DuckDuckGoSearchTool but beware that it has a rate limit.

In [2]:
import os
from PIL import Image
from smolagents import CodeAgent, GoogleSearchTool, InferenceClientModel, VisitWebpageTool

model = InferenceClientModel(model_id='Qwen/Qwen3-Coder-30B-A3B-Instruct')

In [3]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe. 

Also give me some supercar factories with the same cargo plane transfer time."""

In [4]:
agent = CodeAgent(
    model=model,
    tools=[GoogleSearchTool("serpapi"), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20
)

In [5]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│                                                                                                                 │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen3-Coder-30B-A3B-Instruct ──────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_locations = web_search("Batman filming locations worldwide")                                              
  print(batman_locations)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results
0. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

The Batman | 2022 · Merseyside · London · Scotland · Bedfordshire · Hertfordshire · Essex · Surrey · Chicago ...

1. [12 Batman Movie Locations You Can 
Visit!](https://www.travelandleisureasia.com/sea/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023
Source: Travel and Leisure Asia

Most of the filming of Batman movies is done in the Warner Bros studios and across the US, including New York and 
Pittsburgh. You will notice ...

2. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations ; Necropolis Cemetery, Glasgow, Scotland, UK. (Batman and Selina leaving the cemetery) · 88 ; St.
George's Hall, Liverpool, England, UK. ( ...

3. [Category:Film Locations - Batman Wiki](https://batman.fandom.com/wiki/Category:Film_Locations)
Source: Batman Wiki

Category:Batman (1966 Movie) Locations · Category:Batman (1989 film) Locations · Category:Batman Forever Locations 
· Category:Batman Returns Locations.

4. [Exploring the locations of BATMAN 
BEGINS](https://www.facebook.com/DCUnitedKingdom/posts/exploring-the-locations-of-batman-begins-filmtourismus/11914
61673009725/)
Source: Facebook · DC Comics

The film, which was primarily shot in Iceland and Chicago, relied on traditional stunts and miniatures, while 
computer- generated imagery was ...

5. [Batman Movies and Filming Locations in Los 
Angeles](https://www.traveltodayla.com/post/batman-movies-in-la?srsltid=AfmBOoqLBgtiPKLVqqgkXphzTXfkQ_ctKZ2NREhxbJ3
8qpJXU2ImjNWU)
Date published: Apr 20, 2023
Source: LA Today

The legendary Batcave is found in Bronson Canyon, at the Bronson Caves. This is a section of the expansive Griffith
Park in Los Angeles that is ...

6. [What cities in America and anywhere in the world do you 
...](https://www.reddit.com/r/batman/comments/1d1t88q/what_cities_in_america_and_anywhere_in_the_world/)
Source: Reddit · r/batman

Glasgow and Liverpool were used in the shoot for The Batman. I'd avoid Chicago, as that looks more like Metropolis 
than Gotham.

7. [Batman Begins filming locations](https://moviemaps.org/movies/1r)
Source: MovieMaps

Batman Begins was filmed in London in the United Kingdom, Chicago in the United States of America, and in Iceland.

8. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

Guide to the all the filming locations of The Batman - The 2022 movie with Robert Pattison filmed in Chicago and 
The UK - The City and the Wayne House.

9. [Where Was The Batman Filmed?](https://git.flexsim.com/blog/where-was-the-batman-filmed-1764807258)
Date published: Dec 4, 2025
Source: FlexSim

The primary stomping ground for The Batman was London, England . Yep, the Big Smoke transformed into the 
ever-gloomy Gotham. You'll recognize ...

Out: None

[Step 1: Duration 4.77 seconds| Input tokens: 2,324 | Output tokens: 97]

...

[Step 5: Duration 2.58 seconds| Input tokens: 42,786 | Output tokens: 1,869]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 6 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({                                                                                                   
      "batman_locations_dataframe": df_batman_locations,                                                           
      "supercar_factories_dataframe": df_supercar_factories,                                                       
      "similar_time_factories": similar_time_factories                                                             
  })                                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: {'batman_locations_dataframe':             Location           Coordinates  \
0         London, UK    (51.5074, -0.1278)   
1       Chicago, USA   (41.8781, -87.6298)   
2  Glasgow, Scotland    (55.8642, -4.2518)   
3      Liverpool, UK    (53.4084, -2.9914)   
4    Berlin, Germany       (52.52, 13.405)   
5            Iceland   (64.9631, -19.0208)   
6   Los Angeles, USA  (34.0522, -118.2437)   
7      New York, USA    (40.7128, -74.006)   

                                        Description  Distance_km  \
0                 The Batman 2022 filming locations      5570.22   
1                 The Batman 2022 filming locations      1144.29   
2                 The Batman 2022 filming locations      5181.75   
3                 The Batman 2022 filming locations      5323.84   
4         Batman Begins filming location (possibly)      6385.00   
5                   Batman Begins filming locations      4357.89   
6          Batman Begins, The Dark Knight locations      3935.75   
7  Batman Begins, The Dark Knight filming locations         0.00   

   Travel_time_hours  Travel_time_days  
0               7.43             0.309  
1               1.53             0.064  
2               6.91             0.288  
3               7.10             0.296  
4               8.51             0.355  
5               5.81             0.242  
6               5.25             0.219  
7               0.00             0.000  , 'supercar_factories_dataframe':                                      
Factory          Coordinates  \
0                 Ferrari - Maranello, Italy   (44.8323, 10.9061)   
1                       McLaren - Woking, UK   (51.3242, -0.5783)   
2                 Bugatti - Molsheim, France    (48.4026, 7.4214)   
3  Lamborghini - Sant'Agata Bolognese, Italy   (44.5112, 11.2513)   
4               Porsche - Stuttgart, Germany    (48.7767, 9.1759)   
5        Pagani - San Severino Marche, Italy   (43.2069, 13.2911)   
6             Koenigsegg - Ängelholm, Sweden   (56.1987, 14.1625)   
7                Bugatti - Indianapolis, USA  (39.7817, -86.1419)   
8           Tesla - Fremont, California, USA  (37.5015, -122.022)   
9                    Renault - Flins, France    (48.7588, 2.7162)   

                      Description  Distance_km  Travel_time_hours  \
0            Ferrari headquarters      6616.07               8.82   
1                 McLaren factory      5546.94               7.40   
2                 Bugatti factory      6199.19               8.27   
3             Lamborghini factory      6657.24               8.88   
4                 Porsche factory      6298.05               8.40   
5                  Pagani factory      6871.35               9.16   
6              Koenigsegg factory      6255.31               8.34   
7       Bugatti facilities in USA      1034.34               1.38   
8               Tesla Gigafactory      4106.75               5.48   
9  Renault manufacturing facility      5866.03               7.82   

   Travel_time_days  
0             0.368  
1             0.308  
2             0.344  
3             0.370  
4             0.350  
5             0.382  
6             0.348  
7             0.057  
8             0.228  
9             0.326  , 'similar_time_factories':                             Factory          Coordinates  \
1              McLaren - Woking, UK   (51.3242, -0.5783)   
7       Bugatti - Indianapolis, USA  (39.7817, -86.1419)   
8  Tesla - Fremont, California, USA  (37.5015, -122.022)   
9           Renault - Flins, France    (48.7588, 2.7162)   

                      Description  Distance_km  Travel_time_hours  \
1                 McLaren factory      5546.94               7.40   
7       Bugatti facilities in USA      1034.34               1.38   
8               Tesla Gigafactory      4106.75               5.48   
9  Renault manufacturing facility      5866.03               7.82   

   Travel_time_days  
1             0.308  
7             0.057  
8             0.228  


[Step 6: Duration 1.72 seconds| Input tokens: 57,980 | Output tokens: 1,946]

We could already improve this a bit by throwing in some dedicated planning steps, and adding more prompting.

Planning steps allow the agent to think ahead and plan its next steps, which can be useful for more complex tasks.

In [6]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│                                                                                                                 │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen3-Coder-30B-A3B-Instruct ──────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
### 1.1. Facts given in the task
- Current location: Gotham, 40.7128° N, 74.0060° W
- Need to find Batman filming locations worldwide
- Need to calculate cargo plane travel time from Gotham to each location
- Need to identify supercar factories with similar travel times

### 1.2. Facts to look up
- Batman filming locations worldwide (list of locations with coordinates)
- Scarp factory locations worldwide (list of locations with coordinates)
- Cargo plane cruising speed (typically around 750 km/h)
- Geographic coordinates for each Batman filming location
- Geographic coordinates for each supercar factory location

### 1.3. Facts to derive
- Travel time calculation for each Batman filming location using great-circle distance and speed
- Travel time calculation for each supercar factory using great-circle distance and speed
- Filter supercar factories based on similar travel times to Batman locations
- Create pandas dataframe with Batman locations and their travel times
- Create pandas dataframe with supercar factories and their travel times


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Search for Batman filming locations                                                                            
  batman_locations = web_search("Batman filming locations around the world")                                       
  print("Batman filming locations found:")                                                                         
  print(batman_locations)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Batman filming locations found:
## Search Results
0. [12 Batman Movie Locations You Can 
Visit!](https://www.travelandleisureasia.com/sea/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023
Source: Travel and Leisure Asia

One of the many IMDb-listed filming locations for Reeves' The Batman is the iconic St. George's Hall in Liverpool, 
England. Though we have seen ...

1. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

Film locations for The Batman (2022) in Liverpool, London, Glasgow and Chicago.

2. [What cities in America and anywhere in the world do you 
...](https://www.reddit.com/r/batman/comments/1d1t88q/what_cities_in_america_and_anywhere_in_the_world/)
Source: Reddit · r/batman

Glasgow and Liverpool were used in the shoot for The Batman. I'd avoid Chicago, as that looks more like Metropolis 
than Gotham.

3. [Exploring the locations of BATMAN 
BEGINS](https://www.facebook.com/DCUnitedKingdom/posts/exploring-the-locations-of-batman-begins-filmtourismus/11914
61673009725/)
Source: Facebook · DC Comics

The film, which was primarily shot in Iceland and Chicago, relied on traditional stunts and miniatures, while 
computer- generated imagery was ...

4. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations ; Necropolis Cemetery, Glasgow, Scotland, UK. (Batman and Selina leaving the cemetery) · 88 ; St.
George's Hall, Liverpool, England, UK. ( ...

5. [Category:Film Locations - Batman Wiki](https://batman.fandom.com/wiki/Category:Film_Locations)
Source: Batman Wiki

A · Arctic World · Arkham Asylum (Nolanverse) · Arkham Asylum (Schumacherverse) · Arkham State Hospital ...

6. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

Guide to the all the filming locations of The Batman - The 2022 movie with Robert Pattison filmed in Chicago and 
The UK - The City and the Wayne House.

7. [Batman Begins Filming Locations: Complete Guide to 
...](https://giggster.com/guide/movie-location/where-was-batman-begins-filmed)
Source: Giggster

Batman Begins was filmed in various locations across the globe, with most scenes shot in the United Kingdom, 
specifically in London, Buckinghamshire, and Essex.

8. [Batman Movies and Filming Locations in Los 
Angeles](https://www.traveltodayla.com/post/batman-movies-in-la?srsltid=AfmBOoqqx73uif_LPkH_xca0uJWWqWb4CEvrx0TKDTU
TY0ua9wpxQGNc)
Date published: Apr 20, 2023
Source: LA Today

From the Batcave to Wayne Manor, these spots offer a unique way to immerse yourself in the world of the Dark Knight
so get ready to add some superhero fun to ...

9. [Where The Batman Filmed (All Gotham Real-Life 
Locations)](https://screenrant.com/where-was-the-batman-movie-filmed/)
Date published: Mar 4, 2022
Source: Screen Rant

The Batman filmed all over the world to bring a new version of Gotham to life and went to some prominent real-life 
locations to achieve the right look.

Out: None

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Let's present the final results clearly                                                                        
                                                                                                                   
  print("=== BATMAN FILMING LOCATIONS DATAFRAME ===")                                                              
  print("This includes all major Batman filming locations worldwide with travel times from Gotham (40.7128° N,     
  74.0060° W) by cargo plane (750 km/h)")                                                                          
  print()                                                                                                          
  print(batman_complete_df.sort_values(by="Travel Time (hours)"))                                                  
                                                                                                                   
  print("\n=== SUPERCAR FACTORIES WITH SIMILAR TRAVEL TIMES ===")                                                  
  print("The following supercar factories have travel times comparable to Batman filming locations:")              
                                                                                                                   
  # Create matching pairs showing Batman locations and matching supercar factories                                 
  print("\nShort travel times (0-5 hours) matches:")                                                               
  short_matches = [(batman_complete_df.iloc[0]["Location"], "Gotham City"),                                        
                   (batman_complete_df.iloc[1]["Location"], "Newark"),                                             
                   (batman_complete_df.iloc[2]["Location"], "Pittsburgh"),                                         
                   (batman_complete_df.iloc[3]["Location"], "Chicago")]                                            
                                                                                                                   
  print("- Batman locations with very short travel times to Gotham:")                                              
  for loc in short_matches:                                                                                        
      print(f"  * {loc[0]} -> {loc[1]}")                                                                           
                                                                                                                   
  print("\n- Matching supercar factories with similar short travel times (0-5 hours):")                            
  short_supercar_match = supercar_df[supercar_df['Travel Time (hours)'] <= 5]                                      
  for _, row in short_supercar_match.iterrows():                                                                   
      print(f"  * {row['Factory']} ({row['Location']}, {row['Country']}) - {row['Travel Time (hours)']} hours")    
                                                                                                                   
  print("\nMedium travel times (5-15 hours) matches:")                                                             
  medium_matches = [                                                                                               
      (batman_complete_df.iloc[4]["Location"], "Los Angeles"),                                                     
      (batman_complete_df.iloc[5]["Location"], "Warner Bros Studios"),                                             
      (batman_complete_df.iloc[[38;2;174;129;255;48;2;39;40

Execution logs:
=== BATMAN FILMING LOCATIONS DATAFRAME ===
This includes all major Batman filming locations worldwide with travel times from Gotham (40.7128° N, 74.0060° W) 
by cargo plane (750 km/h)

                        Location             City  Country  Latitude  \
4                    Gotham City    New York City      USA   40.7128   
14                        Newark           Newark      USA   40.7306   
9                     Pittsburgh       Pittsburgh      USA   40.4406   
3                        Chicago          Chicago      USA   41.8781   
8                    Los Angeles      Los Angeles      USA   34.0522   
6            Warner Bros Studios          Burbank      USA   34.1439   
7   The 20th Century Fox Studios         Westwood      USA   34.0902   
5       Svinafellsjokull Glacier          Iceland  Iceland   64.1333   
1            Necropolis Cemetery          Glasgow       UK   55.8681   
0              St. George's Hall        Liverpool       UK   53.4089   
13               Buckinghamshire  Buckinghamshire       UK   51.8333   
11                  Bedfordshire     Bedfordshire       UK   52.1333   
2              Millennium Bridge           London       UK   51.5055   
12                         Essex            Essex       UK   51.7500   
10                     Hong Kong        Hong Kong    China   22.3193   

    Longitude  Travel Time (hours)  Travel Time (days)  
4    -74.0060                 1.00                0.04  
14   -74.1724                 1.02                0.04  
9    -79.9959                 1.74                0.07  
3    -87.6298                 2.68                0.11  
8   -118.2437                 6.77                0.28  
6   -118.3866                 6.78                0.28  
7   -118.3956                 6.79                0.28  
5    -16.3333                 7.56                0.32  
1     -4.3220                 8.59                0.36  
0     -2.9838                 8.81                0.37  
13    -0.7667                 9.09                0.38  
11    -0.4500                 9.11                0.38  
2     -0.1095                 9.17                0.38  
12     0.5000                 9.22                0.38  
10   114.1694                20.00                0.83  

=== SUPERCAR FACTORIES WITH SIMILAR TRAVEL TIMES ===
The following supercar factories have travel times comparable to Batman filming locations:

Short travel times (0-5 hours) matches:
- Batman locations with very short travel times to Gotham:
  * St. George's Hall -> Gotham City
  * Necropolis Cemetery -> Newark
  * Millennium Bridge -> Pittsburgh
  * Chicago -> Chicago

- Matching supercar factories with similar short travel times (0-5 hours):
  * Tesla (Austin, USA) - 4.56 hours
  * Ford Mustang (Dearborn, USA) - 2.13 hours
  * Honda NSX (Maryland, USA) - 1.41 hours
  * Chevrolet Corvette (Detroit, USA) - 2.13 hours

Medium travel times (5-15 hours) matches:

- Batman locations with medium travel times (5-15 hours):
  * Gotham City -> Los Angeles
  * Svinafellsjokull Glacier -> Warner Bros Studios
  * Warner Bros Studios -> 20th Century Fox Studios
  * The 20th Century Fox Studios -> Svinafellsjokull Glacier
  * Los Angeles -> Necropolis Cemetery

- Matching supercar factories with similar medium travel times (5-15 hours):
  * Ferrari (Maranello, Italy) - 10.73 hours
  * Lamborghini (Santa Croce sull'Arno, Italy) - 10.73 hours
  * Bugatti (Molsheim, France) - 10.11 hours
  * McLaren (Woking, UK) - 9.14 hours
  * Pagani (Sanremo, Italy) - 10.41 hours
  * Koenigsegg (Åtvidaberg, Sweden) - 10.17 hours
  * Porsche (Zuffenhausen, Germany) - 10.25 hours
  * Bugatti (Molsheim, France) - 10.11 hours
  * Mercedes-AMG (Böblingen, Germany) - 10.23 hours
  * Aston Martin (Gaydon, UK) - 8.96 hours
  * Lotus (Hethel, UK) - 9.2 hours
  * Rolls-Royce (Goodwood, UK) - 9.16 hours

=== SUMMARY ===
Total Batman filming locations identified: 15
Total supercar factories analyzed: 18
Batman locations sorted by travel time f

Final answer: {'batman_filming_locations': [{'Location': "St. George's Hall", 'City': 'Liverpool', 'Country': 'UK',
'Latitude': 53.4089, 'Longitude': -2.9838, 'Travel Time (hours)': 8.81, 'Travel Time (days)': 0.37}, {'Location': 
'Necropolis Cemetery', 'City': 'Glasgow', 'Country': 'UK', 'Latitude': 55.8681, 'Longitude': -4.322, 'Travel Time 
(hours)': 8.59, 'Travel Time (days)': 0.36}, {'Location': 'Millennium Bridge', 'City': 'London', 'Country': 'UK', 
'Latitude': 51.5055, 'Longitude': -0.1095, 'Travel Time (hours)': 9.17, 'Travel Time (days)': 0.38}, {'Location': 
'Chicago', 'City': 'Chicago', 'Country': 'USA', 'Latitude': 41.8781, 'Longitude': -87.6298, 'Travel Time (hours)': 
2.68, 'Travel Time (days)': 0.11}, {'Location': 'Gotham City', 'City': 'New York City', 'Country': 'USA', 
'Latitude': 40.7128, 'Longitude': -74.006, 'Travel Time (hours)': 1.0, 'Travel Time (days)': 0.04}, {'Location': 
'Svinafellsjokull Glacier', 'City': 'Iceland', 'Country': 'Iceland', 'Latitude': 64.1333, 'Longitude': -16.3333, 
'Travel Time (hours)': 7.56, 'Travel Time (days)': 0.32}, {'Location': 'Warner Bros Studios', 'City': 'Burbank', 
'Country': 'USA', 'Latitude': 34.1439, 'Longitude': -118.3866, 'Travel Time (hours)': 6.78, 'Travel Time (days)': 
0.28}, {'Location': 'The 20th Century Fox Studios', 'City': 'Westwood', 'Country': 'USA', 'Latitude': 34.0902, 
'Longitude': -118.3956, 'Travel Time (hours)': 6.79, 'Travel Time (days)': 0.28}, {'Location': 'Los Angeles', 
'City': 'Los Angeles', 'Country': 'USA', 'Latitude': 34.0522, 'Longitude': -118.2437, 'Travel Time (hours)': 6.77, 
'Travel Time (days)': 0.28}, {'Location': 'Pittsburgh', 'City': 'Pittsburgh', 'Country': 'USA', 'Latitude': 
40.4406, 'Longitude': -79.9959, 'Travel Time (hours)': 1.74, 'Travel Time (days)': 0.07}, {'Location': 'Hong Kong',
'City': 'Hong Kong', 'Country': 'China', 'Latitude': 22.3193, 'Longitude': 114.1694, 'Travel Time (hours)': 20.0, 
'Travel Time (days)': 0.83}, {'Location': 'Bedfordshire', 'City': 'Bedfordshire', 'Country': 'UK', 'Latitude': 
52.1333, 'Longitude': -0.45, 'Travel Time (hours)': 9.11, 'Travel Time (days)': 0.38}, {'Location': 'Essex', 
'City': 'Essex', 'Country': 'UK', 'Latitude': 51.75, 'Longitude': 0.5, 'Travel Time (hours)': 9.22, 'Travel Time 
(days)': 0.38}, {'Location': 'Buckinghamshire', 'City': 'Buckinghamshire', 'Country': 'UK', 'Latitude': 51.8333, 
'Longitude': -0.7667, 'Travel Time (hours)': 9.09, 'Travel Time (days)': 0.38}, {'Location': 'Newark', 'City': 
'Newark', 'Country': 'USA', 'Latitude': 40.7306, 'Longitude': -74.1724, 'Travel Time (hours)': 1.02, 'Travel Time 
(days)': 0.04}], 'supercar_factories': [{'Factory': 'Ferrari', 'Location': 'Maranello', 'Country': 'Italy', 
'Latitude': 44.5748, 'Longitude': 10.9591, 'Travel Time (hours)': 10.73, 'Travel Time (days)': 0.45}, {'Factory': 
'Lamborghini', 'Location': "Santa Croce sull'Arno", 'Country': 'Italy', 'Latitude': 43.8189, 'Longitude': 10.4279, 
'Travel Time (hours)': 10.73, 'Travel Time (days)': 0.45}, {'Factory': 'Bugatti', 'Location': 'Molsheim', 
'Country': 'France', 'Latitude': 48.5948, 'Longitude': 7.6922, 'Travel Time (hours)': 10.11, 'Travel Time (days)': 
0.42}, {'Factory': 'McLaren', 'Location': 'Woking', 'Country': 'UK', 'Latitude': 51.3285, 'Longitude': -0.5533, 
'Travel Time (hours)': 9.14, 'Travel Time (days)': 0.38}, {'Factory': 'Pagani', 'Location': 'Sanremo', 'Country': 
'Italy', 'Latitude': 43.8415, 'Longitude': 7.3316, 'Travel Time (hours)': 10.41, 'Travel Time (days)': 0.43}, 
{'Factory': 'Koenigsegg', 'Location': 'Åtvidaberg', 'Country': 'Sweden', 'Latitude': 57.4034, 'Longitude': 15.0706,
'Travel Time (hours)': 10.17, 'Travel Time (days)': 0.42}, {'Factory': 'Porsche', 'Location': 'Zuffenhausen', 
'Country': 'Germany', 'Latitude': 48.7469, 'Longitude': 9.3328, 'Travel Time (hours)': 10.25, 'Travel Time (days)':
0.43}, {'Factory': 'Bugatti', 'Location': 'Molsheim', 'Country': 'France', 'Latitude': 48.5948, 'Longitude': 
7.6922, 'Travel Time (hours)': 10

[Step 7: Duration 6.96 seconds| Input tokens: 57,428 | Output tokens: 5,227]

{'batman_filming_locations': [{'Location': "St. George's Hall", 'City': 'Liverpool', 'Country': 'UK', 'Latitude': 53.4089, 'Longitude': -2.9838, 'Travel Time (hours)': 8.81, 'Travel Time (days)': 0.37}, {'Location': 'Necropolis Cemetery', 'City': 'Glasgow', 'Country': 'UK', 'Latitude': 55.8681, 'Longitude': -4.322, 'Travel Time (hours)': 8.59, 'Travel Time (days)': 0.36}, {'Location': 'Millennium Bridge', 'City': 'London', 'Country': 'UK', 'Latitude': 51.5055, 'Longitude': -0.1095, 'Travel Time (hours)': 9.17, 'Travel Time (days)': 0.38}, {'Location': 'Chicago', 'City': 'Chicago', 'Country': 'USA', 'Latitude': 41.8781, 'Longitude': -87.6298, 'Travel Time (hours)': 2.68, 'Travel Time (days)': 0.11}, {'Location': 'Gotham City', 'City': 'New York City', 'Country': 'USA', 'Latitude': 40.7128, 'Longitude': -74.006, 'Travel Time (hours)': 1.0, 'Travel Time (days)': 0.04}, {'Location': 'Svinafellsjokull Glacier', 'City': 'Iceland', 'Country': 'Iceland', 'Latitude': 64.1333, 'Longitude': -16.3

Thanks to these quick changes, we obtained a much more concise report by simply providing our agent a detailed prompt, and giving it planning capabilities!

The model’s context window is quickly filling up. So if we ask our agent to combine the results of detailed search with another, it will be slower and quickly ramp up tokens and costs.

➡️ We need to improve the structure of our system.

✌️ Splitting the task between two agents
Multi-agent structures allow to separate memories between different sub-tasks, with two great benefits:

Each agent is more focused on its core task, thus more performant
Separating memories reduces the count of input tokens at each step, thus reducing latency and cost.
Let’s create a team with a dedicated web search agent, managed by another agent.

The manager agent should have plotting capabilities to write its final report: so let us give it access to additional imports, including plotly, and geopandas + shapely for spatial plotting.

In [7]:
model = InferenceClientModel(model_id='Qwen/Qwen3-Coder-30B-A3B-Instruct')

In [8]:
web_agent = CodeAgent(
    model=model,
    tools=[
        GoogleSearchTool(provider="serpapi"),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name='web_agent',
    description="Browse the web to find information",
    verbosity_level=0,
    max_steps=10,
)

The manager agent will need to do some mental heavy lifting.

So we give it the stronger model DeepSeek-R1, and add a planning_interval to the mix.

In [9]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel

def check_reasoning_and_plot(final_answer, agent_memory):
    multimodal_model = InferenceClientModel(max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True

manager_agent = CodeAgent(
    model=InferenceClientModel(max_tokens=8096),
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

Let us inspect what this team looks like:

In [10]:
manager_agent.visualize()

CodeAgent | Qwen/Qwen3-Next-80B-A3B-Thinking
├── ✅ Authorized imports: ['geopandas', 'plotly', 'shapely', 'json', 'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | Qwen/Qwen3-Coder-30B-A3B-Instruct
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browse the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a google web search for  │ query (`string`): The search      │
            │                             │ your query then returns a string  │ query to perform.                 │
            │                             │ of the top search results.        │ filter_year (`integer`):          │
            │                             │                                   │ Optionally restrict results to a  │
            │                             │                                   │ certain year                      │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude, longitude) for the     │
            │                             │ Earth using great-circle          │ starting point                    │
            │                             │ dist

In [11]:
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W).                                                                             │
│ Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in     │
│ total.                                                                                                          │
│ Represent this as spatial map of the world, with the locations represented as scatter points with a color that  │
│ depends on the travel time, and save it to saved_map.png!                                                       │
│                                                                                                                 │
│ Here's an example of how to plot and return a map:                                                              │
│ import plotly.express as px                                                                                     │
│ df = px.data.carshare()                                                                                         │
│ fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,      │
│      color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)                                    │
│ fig.show()                                                                                                      │
│ fig.write_image("saved_image.png")                                                                              │
│ final_answer(fig)                                                                                               │
│                                                                                                                 │
│ Never try to process strings using code: when you have a string to read, just print it and you'll see it.       │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen3-Next-80B-A3B-Thinking ───────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey
### 1.1. Facts given in the task
- Gotham coordinates: 40.7128° N, 74.0060° W
- Cargo plane cruising speed defaults to 750 km/h (as specified in the `calculate_cargo_travel_time` function 
documentation)

### 1.2. Facts to look up
- Batman filming locations with specific latitude and longitude coordinates (e.g., cities or landmarks used in 
Batman films like "The Dark Knight", "Batman Begins", etc.)  
  Source: Web search via `web_agent` using queries like "Batman movie filming locations coordinates", "The Dark 
Knight Chicago filming sites", "Batman Gotham filming locations real places"
- Supercar factory headquarters coordinates (e.g., Ferrari, Lamborghini, McLaren, Aston Martin, Porsche, Bugatti)  
  Source: Web search via `web_agent` using queries like "Ferrari factory coordinates", "Lamborghini headquarters 
latitude longitude", "McLaren production site coordinates", etc.

### 1.3. Facts to derive
- Travel time from each Batman filming location and supercar factory to Gotham (40.7128° N, 74.0060° W) using 
great-circle distance calculation with 750 km/h cruising speed  
  Reasoning: Requires applying the `calculate_cargo_travel_time` function to each location's coordinates to compute
travel time based on Earth's curvature.

## 2. Plan
1. Use web_agent to search for and collect Batman filming locations with precise latitude and longitude coordinates
by querying specific Batman movie filming sites.
2. Use web_agent to search for and collect supercar factory headquarters coordinates by querying specific brand 
names and their production sites.
3. Combine the collected locations to ensure at least six total points (mix of Batman filming sites and supercar 
factories).
4. For each location, calculate the cargo travel time to Gotham using the calculate_cargo_travel_time function.
5. Create a Plotly scatter_map plot with all locations as points colored by travel time and save the figure as 
"saved_map.png".
6. Return the final answer with the saved map.

```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
<code>                                                                                                             
batman_locations = web_agent(task="Find all Batman movie filming locations with precise latitude and longitude     
coordinates, including specific scenes from movies like The Dark Knight, Batman Begins, etc. Provide location name,
movie title, and coordinates.", additional_args={})                                                                
print(batman_locations)                                                                                            
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_locations = web_agent(task="Find all Batman movie filming locations with precise latitude and longitude   
  coordinates, including specific scenes from movies like The Dark Knight, Batman Begins, etc. Provide location    
  name, movie title, and coordinates.", additional_args={})                                                        
  print(batman_locations)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

Check check_reasoning_and_plot failed with error: check_reasoning_and_plot() got an unexpected keyword argument 
'agent'

[Step 15: Duration 12.02 seconds| Input tokens: 170,815 | Output tokens: 57,657]

Reached max steps.

[Step 16: Duration 11.27 seconds| Input tokens: 187,929 | Output tokens: 59,422]

'https://staticmap.openstreetmap.de/staticmap.php?center=40.7128,-74.0060&zoom=1&size=800x600&markers=64.007885,-16.873446,blue&markers=52.948004,-1.20958,green&markers=53.3807,-2.9743,green&markers=44.52639,10.86667,red&markers=51.345833,-0.548333,green&markers=48.408611,7.538889,red'